In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import math
import xlwt
from pathlib import Path

In [4]:

BASE_DIR = Path().absolute()
DIR = BASE_DIR/'Results'
DIR


WindowsPath('c:/Users/fan_shuyuan/Desktop/科研ing/工艺流程优化/ASBO-V6/Results')

In [5]:
*

SyntaxError: invalid syntax (796671574.py, line 1)

In [ ]:
sol = 'ASBO'
case_name = 'Case2'
runs_number = 100
list_realvalue = []

def func(x1,x2):
    x = np.array([x1,x2])
    obj, c1, = Case2(x)
    cons = [c1]
    return -obj,c1

nc_lb = np.array([-1e6])
nc_ub = np.array([0,])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 10),'x2': (0, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)


for i in range(runs_number):
    optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
    optimizer.maximize(init_points=100, n_iter=50,p_hyper=[5, 2, 0.78, 0.95,1e-5],acquisition_function=acquisition_function)
    print(optimizer.max)
    list_realvalue.extend([optimizer.max['target']])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    for i in range(len(list_realvalue)):
        res = list_realvalue[i]
        sheet1.write(i,0,res)
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


{'target': 268.6430578187194, 'params': {'x1': 3.247533685062293, 'x2': 0.0652332966212359}, 'constraint': -0.017350971511890023}
{'target': 268.6489479762007, 'params': {'x1': 3.2706788920182683, 'x2': 0.05459316706727202}, 'constraint': -0.019803675265670506}
{'target': 268.6338841913747, 'params': {'x1': 3.241863008162683, 'x2': 0.0681056596647866}, 'constraint': -0.0169978137702822}
{'target': 268.6474921520931, 'params': {'x1': 3.255067050007794, 'x2': 0.061681096850204044}, 'constraint': -0.018406015474599968}
{'target': 268.6540207016272, 'params': {'x1': 3.255352399334486, 'x2': 0.061333746131681195}, 'constraint': -0.017528886538865862}
{'target': 268.6374090719724, 'params': {'x1': 3.2545046014948102, 'x2': 0.0622725657241371}, 'constraint': -0.019735979073240983}
{'target': 268.64904619819055, 'params': {'x1': 3.264179967442051, 'x2': 0.05751778695448554}, 'constraint': -0.019431473031739088}
{'target': 268.6406415878431, 'params': {'x1': 3.2519879798890083, 'x2': 0.06330105

In [ ]:
sol = 'trust-constr'
list_realvalue = []
case_name = 'Case2'
runs_number = 100
initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1= Case2(x)
    cons = [c1]
    list_realvalue.extend([[obj]+cons])
    return obj,c1

def recyc(x):
    nc_lb = np.array([-1e6])
    nc_ub = np.array([0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5,0] 
    ub = [10,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
    print(result.fun)
    initial_x.append(result.x)
    fun_r.append(result.fun)
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    f = xlwt.Workbook('encoding = utf-8') 
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True) 
    for i in range(len(initial_x)):
        x_values = initial_x[i].tolist()
        sheet1.write(i,0,x_values[0]) 
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,fun_r[i])
        sheet1.write(i,3,nfev_r[i])
        sheet1.write(i,4,success_r[i])
        sheet1.write(i,5,message_r[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

x_random = initial_random(lb=[-5,0],ub=[10,15])
from itertools import product
for combination in product(x_random):
    recyc(combination)


-268.78838103316497
-200.66282779946073
-268.7883810416733
-200.66282780022857
-268.7883808282727
-268.7883810361163
-200.66282784363506
-200.66282779605527
`x0` violates bound constraints.
`x0` violates bound constraints.
`x0` violates bound constraints.
`x0` violates bound constraints.
-268.7883808058489
-220.17342701138534
-268.78838104174906
-268.78838080389164
-200.66282780023235
-268.78838104166584
-200.66282779750193
-268.7883810398107
-268.7883808061378
-268.7883808058441
-268.78838080584535
-200.662827797028
-268.7883810405484
-220.17342679988474
-268.7883810416292
-200.66282779756872
-200.6628277997172
-268.7883810333848
-200.6628277995161
-268.7883808058353
-268.7883810416728
-220.17342705214628
-268.7883808061476
-220.17342701084448
-200.6628277991461
-268.7883808058273
-220.17342700991404
-268.78838080412163
-200.6628278006385
-268.7883810331592
-268.78838104164396
-220.1734267998953
-268.788380802891
-268.78838080614764
-268.7883808058443
-200.66282779952945
-225.79308868

In [ ]:
sol = 'slsqp'
list_realvalue = []
case_name = 'Case2'
runs_number = 100
initial_x = []
fun_r = []
nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1= Case2(x)
    cons = [c1]
    list_realvalue.extend([[obj]+cons])
    return obj,c1

def recyc(x):
    nc_lb = np.array([-1e6])
    nc_ub = np.array([0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x
    lb = [-5,0] 
    ub = [10,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print(result.fun)
    initial_x.append(result.x)
    fun_r.append(result.fun)
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    f = xlwt.Workbook('encoding = utf-8') 
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True) 
    for i in range(len(initial_x)):
        x_values = initial_x[i].tolist()
        sheet1.write(i,0,x_values[0]) 
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,fun_r[i])
        sheet1.write(i,3,nfev_r[i])
        sheet1.write(i,4,success_r[i])
        sheet1.write(i,5,message_r[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

x_random = initial_random(lb=[-5,0],ub=[10,15])
from itertools import product
for combination in product(x_random):
    recyc(combination)


-268.7885046713122
-268.78850467678114
-268.7885048274905
-268.7885054032581
-268.78850480881397
-268.78850467530833
-200.66314216194593
-268.78851507991527
-268.7885048892059
-220.17428280328667
-268.7885047182179
-268.7885048738483
-200.66314217785478
-268.78850466132735
-200.66314407376902
-268.7885049436456
-268.78850467698624
-268.78850467511734
-268.7885046803757
-268.78850469774176
-220.17428277272273
-220.17428277275403
-268.78850467571635
-268.78852185078625
-200.6631421784669
-268.7885049259929
-268.78850467127177
-268.78850499697467
-268.78850569048086
-220.1742827735918
-268.78850500185337
-220.17428277273177
-268.78850467186237
-268.7885048313142
-220.17428278238242
-268.7885046752282
-200.66314217779833
-268.78850467083834
-268.78850468998434
-220.17428277329225
-268.78850467183156
-268.7885046762167
-268.78850467539
-268.7885046781623
-220.17428277338746
-268.7885049532575
-268.78850511286294
-200.66314218737364
-268.78850467847
-268.78850467646504
-220.17428277278648
-2

In [ ]:
sol = 'CBO'
case_name = 'Case2'
list_realvalue = []
runs_number = 100
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

def func_obj(x1,x2):
    x = np.array([x1,x2])
    obj, c1 = Case2(x)
    cons = [c1]
    return -obj

def func_cons(x1,x2):
    x = np.array([x1,x2])
    obj, c1 = Case2(x)
    return c1

nc_lb = np.array([-1e6])
nc_ub = np.array([0,])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 10),'x2': (0, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

for i in range(runs_number):
    optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
    optimizer.maximize(init_points=100, n_iter=50,acquisition_function=acquisition_function)
    print(optimizer.max)
    list_realvalue.extend([optimizer.max['target']])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    for i in range(len(list_realvalue)):
        res = list_realvalue[i]
        sheet1.write(i,0,res)
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


{'target': 254.51117417656474, 'params': {'x1': 3.797694022151248, 'x2': 0.3016131928124033}, 'constraint': -0.3061983505082946}
{'target': 263.0013363474833, 'params': {'x1': 3.3957547308121363, 'x2': 0.1883397023867378}, 'constraint': -0.6958078432812682}
{'target': 252.83314556821014, 'params': {'x1': 3.740273327935519, 'x2': 0.3832641277453913}, 'constraint': -0.7678254093955195}
{'target': 240.59523336815826, 'params': {'x1': 2.710525897261629, 'x2': 1.30844053171207}, 'constraint': -1.9634887955332916}
{'target': 230.4210693623198, 'params': {'x1': 2.8254418366743703, 'x2': 1.6229007433084601}, 'constraint': -3.2952743975783445}
{'target': 199.1611327290972, 'params': {'x1': -4.102675448680481, 'x2': 14.474949604054984}, 'constraint': -0.44600181382647186}
{'target': 219.28025657052206, 'params': {'x1': 3.501951569875768, 'x2': 1.693775021807196}, 'constraint': -3.8849011707134693}
{'target': 214.99967883576042, 'params': {'x1': 8.968090295335326, 'x2': 0.3734884132552202}, 'cons

In [ ]:
sol = 'botorch'
case_name = 'Case2'
list_realvalue = []
iterations = []
max_values = []
runs_number = 100
itera = 0
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    global itera
    obj, c1 = Case2(x)
    cons = [c1]
    itera=itera+1
    return -obj,c1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 2
batch_size = 1
n_init = 100 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1e6, -1e6, -1e6, ], dtype=torch.float64)
ub =  torch.tensor([0, 0,0 ], dtype=torch.float64)
bounds = torch.tensor([[-5, 0,], [10, 15]])

def objective(x):
    res = func(x)
    target = res[0]
    c1 = res[1]
    return target,c1
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []

            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1,= combined_tensor[0], combined_tensor[1]
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)

            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)

                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next,  = combined_tensor[0], combined_tensor[1]
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)

                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)

                C_next = torch.cat([C1_next], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                bool_tensor = (C1 >= lb.repeat(len(C1), 1)) & (C1 <= ub.repeat(len(C1), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                if len(Valid_Y)==0:
                    break
                else:
                    list_realvalue.append(max(Valid_Y[0]))
                if len(list_realvalue) == 50:
                    max_value = max(list_realvalue).item()
                    print("Valid_Y",max_value)
                    max_values.append(max_value)
                    f = xlwt.Workbook('encoding = utf-8')
                    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
                    if len(iterations)==0:
                        iterations.append(itera)
                    else:
                        iterations.append(itera-sum(iterations))
                    for i in range(len(iterations)):
                        sheet1.write(i,0,max_values[i])
                        sheet1.write(i,1,iterations[i])
                    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
                    list_realvalue.clear()
                    break
        except Exception as e:
            if len(list_realvalue)==0:
                max_value=None
            else:
                max_value = max(list_realvalue).item()
            print("Attempting another calculation...",max_value)
            max_values.append(max_value)
            f = xlwt.Workbook('encoding = utf-8')
            sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
            if len(iterations)==0:
                iterations.append(itera)
            else:
                iterations.append(itera-sum(iterations))
            for i in range(len(iterations)):
                sheet1.write(i,0,max_values[i])
                sheet1.write(i,1,iterations[i])
            f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')
            list_realvalue.clear()
run_multiple_calculations(num_runs=runs_number)

Attempting another calculation... None
Attempting another calculation... 205.23621900713766
Attempting another calculation... 261.27405964828245
Attempting another calculation... 171.57500429364677
Attempting another calculation... 140.03867965220465
Attempting another calculation... 196.2832248421804
Attempting another calculation... 137.41882430054724
Attempting another calculation... 189.61381090856435
Attempting another calculation... 173.84160213019626
Attempting another calculation... 181.74230011321706
Attempting another calculation... 176.63826575670922
Attempting another calculation... 184.29597445615133
Attempting another calculation... 169.468881374347
Attempting another calculation... 211.48142479689588
Attempting another calculation... 171.6720184723756
Attempting another calculation... 204.3887648224211
Attempting another calculation... 194.04279087793702
Attempting another calculation... 200.59749797376952
Attempting another calculation... 165.1893692869637
Attempting an

In [10]:
sol = 'PSO'
case_name = 'Case2'
list_realvalue = []
iterations=[]
runs_number = 100
itera = 0
from pyswarm import pso

def func(x):
    global itera
    obj, c1 = Case2(x)
    cons = [-c1]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

lb = [-5,0] 
ub = [10,15] 
for num in range(runs_number):
    xopt, fopt = pso(lambda x:func(x)[0], lb, ub,f_ieqcons=lambda x:func(x)[1:2], swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=50, minstep=1e-8,minfunc=1e-8, debug=False)
    print('xopt, fopt',xopt, fopt)
    xopt = np.append(xopt,fopt)
    list_realvalue.extend([xopt])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    if len(iterations)==0:
        iterations.append(itera)
    else:
        iterations.append(itera-sum(iterations))
    for i in range(len(list_realvalue)):
        x_values = list_realvalue[i]
        sheet1.write(i,0,x_values[0])
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,x_values[2])
        sheet1.write(i,3,iterations[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


Stopping search: maximum iterations reached --> 50
xopt, fopt [3.26753402 0.05137807] -268.78739570637515
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.25332228 0.05822601] -268.7742702377176
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.27012859 0.05018376] -268.788174678043
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.27390726 0.04847355] -268.7884668142919
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.26963851 0.05040764] -268.7880776196927
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.27988883 0.04584083] -268.78677058956276
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.25438944 0.0576942 ] -268.7757644631798
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.26684022 0.05170075] -268.7870911026187
Stopping search: maximum iterations reached --> 50
xopt, fopt [3.27436147 0.04827022] -268.7884371253282
Stopping search: maximum iterations reached --> 50
xop

In [ ]:
sol = 'GA'
case_name = 'Case2'
list_realvalue = []
iterations=[]
runs_number = 100
itera = 0
from sko.GA import GA
from sko.operators import crossover

def func(x):
    global itera
    obj, c1= Case2(x)
    cons = [c1]
    itera=itera+1
    return obj,c1

for num in range(runs_number):
    ga = GA(func=lambda x:func(x)[0], n_dim=2, size_pop=100, max_iter=50, lb =[-5,0],ub=[10,15], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[1]])
    ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
    best_x, best_y = ga.run()
    best_res = np.concatenate((best_y, best_x))
    print('best_res',best_res)
    list_realvalue.extend([best_res])
    f = xlwt.Workbook('encoding = utf-8')
    sheet1 = f.add_sheet('sheet1',cell_overwrite_ok=True)
    if len(iterations)==0:
        iterations.append(itera)
    else:
        iterations.append(itera-sum(iterations))
    for i in range(len(list_realvalue)):
        x_values = list_realvalue[i]
        sheet1.write(i,0,x_values[0])
        sheet1.write(i,1,x_values[1])
        sheet1.write(i,2,x_values[2])
        sheet1.write(i,3,iterations[i])
    f.save(DIR/f'{sol}_Results_{case_name}_{runs_number}.xls')


best_res [-246.61427698    2.49408196    1.20596247]
best_res [-263.25585279    2.94957715    0.38673923]
best_res [-2.66693733e+02  3.13199402e+00  1.83650014e-01]
best_res [-2.66234220e+02  3.02200540e+00  2.50701328e-01]
best_res [-2.68714186e+02  3.27738646e+00  4.93928792e-02]
best_res [-2.67485841e+02  3.09360183e+00  1.74767934e-01]
best_res [-2.67681578e+02  3.14010698e+00  1.46601555e-01]
best_res [-2.68023084e+02  3.13009334e+00  1.39735323e-01]
best_res [-263.38651925    2.88450726    0.4138325 ]
best_res [-257.20165626    2.73629292    0.70174082]
best_res [-263.77428149    2.9040233     0.39105082]
best_res [-2.66385162e+02  3.01644418e+00  2.48216708e-01]
best_res [-242.18538253    2.42407837    1.40622223]
best_res [-260.89236483    2.80284389    0.53994091]
best_res [-255.24138293    2.66118855    0.8090441 ]
best_res [-265.24942717    2.97554624    0.30624362]
best_res [-2.68781316e+02  3.28517967e+00  4.36466934e-02]
best_res [-250.50950647    2.56407929    1.02800696